In [32]:
import pandas as pd
import numpy as np
from pathlib import Path
import warnings
warnings.filterwarnings('ignore')

# ML
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.calibration import CalibratedClassifierCV
from sklearn.model_selection import TimeSeriesSplit
from sklearn.metrics import classification_report, accuracy_score, precision_score, recall_score, f1_score
import xgboost as xgb
import lightgbm as lgb
import joblib

# GPU check
import torch
GPU_AVAILABLE = torch.cuda.is_available()

# Paths
BASE_DIR = Path.cwd()
DATA_DIR = BASE_DIR / 'data'
MODEL_DIR = BASE_DIR / 'models' / 'signal_generator_v5'
MODEL_DIR.mkdir(parents=True, exist_ok=True)

print("="*60)
print("🚀 FOREX SIGNAL GENERATOR V5")
print("   Calibrated Probabilities + Time Features")
print("="*60)
print(f"✓ Libraries loaded")
print(f"✓ GPU Available: {GPU_AVAILABLE}")
print(f"✓ Models Directory: {MODEL_DIR}")

🚀 FOREX SIGNAL GENERATOR V5
   Calibrated Probabilities + Time Features
✓ Libraries loaded
✓ GPU Available: True
✓ Models Directory: c:\Users\Acer\Desktop\Forex-Signal-App\models\signal_generator_v5


## 1. Data Loading

In [33]:
# Load data
train_df = pd.read_csv(DATA_DIR / 'EUR_USD_1min.csv')
test_df = pd.read_csv(DATA_DIR / 'EUR_USD_test.csv')

# Rename timestamp to time if needed
if 'timestamp' in train_df.columns:
    train_df.rename(columns={'timestamp': 'time'}, inplace=True)
if 'timestamp' in test_df.columns:
    test_df.rename(columns={'timestamp': 'time'}, inplace=True)

# Convert time to datetime
train_df['time'] = pd.to_datetime(train_df['time'])
test_df['time'] = pd.to_datetime(test_df['time'])

print(f"Train data: {len(train_df):,} rows")
print(f"Test data: {len(test_df):,} rows")
print(f"\nTrain period: {train_df['time'].min()} to {train_df['time'].max()}")
print(f"Test period: {test_df['time'].min()} to {test_df['time'].max()}")

Train data: 1,859,492 rows
Test data: 296,778 rows

Train period: 2019-12-31 16:00:00+00:00 to 2024-12-30 16:00:00+00:00
Test period: 2024-12-31 16:00:00+00:00 to 2025-10-17 06:11:00+00:00


## 2. Feature Engineering (V2 Base + V5 Enhancements)

In [18]:
def add_features_v5(df):
    """V5 Features = V2 Core Features + Time Features + Enhanced Momentum"""
    df = df.copy()
    
    # ==================== TIME FEATURES (NEW in V5) ====================
    df['hour'] = df['time'].dt.hour
    df['day_of_week'] = df['time'].dt.dayofweek
    
    # Trading Sessions (UTC)
    # Asian: 00:00-08:00, London: 08:00-16:00, NY: 13:00-21:00
    df['is_asian'] = ((df['hour'] >= 0) & (df['hour'] < 8)).astype(int)
    df['is_london'] = ((df['hour'] >= 8) & (df['hour'] < 16)).astype(int)
    df['is_ny'] = ((df['hour'] >= 13) & (df['hour'] < 21)).astype(int)
    df['is_overlap'] = ((df['hour'] >= 13) & (df['hour'] < 16)).astype(int)  # London-NY overlap
    
    # Avoid low liquidity times
    df['is_low_liquidity'] = ((df['hour'] >= 21) | (df['hour'] < 2)).astype(int)
    
    # ==================== V2 CORE FEATURES ====================
    # Moving Averages
    for period in [5, 10, 20, 50, 100, 200]:
        df[f'sma_{period}'] = df['close'].rolling(period).mean()
        df[f'ema_{period}'] = df['close'].ewm(span=period, adjust=False).mean()
    
    # MA Crossovers
    df['sma_5_20_cross'] = (df['sma_5'] > df['sma_20']).astype(int)
    df['sma_20_50_cross'] = (df['sma_20'] > df['sma_50']).astype(int)
    df['ema_10_50_cross'] = (df['ema_10'] > df['ema_50']).astype(int)
    df['golden_cross'] = (df['sma_50'] > df['sma_200']).astype(int)
    
    # Price vs MAs
    df['price_vs_sma20'] = (df['close'] - df['sma_20']) / df['sma_20'] * 100
    df['price_vs_sma50'] = (df['close'] - df['sma_50']) / df['sma_50'] * 100
    df['price_above_all_ma'] = ((df['close'] > df['sma_20']) & 
                                 (df['close'] > df['sma_50']) & 
                                 (df['close'] > df['ema_20'])).astype(int)
    
    # RSI
    for period in [7, 14, 21]:
        delta = df['close'].diff()
        gain = delta.where(delta > 0, 0).rolling(period).mean()
        loss = (-delta.where(delta < 0, 0)).rolling(period).mean()
        rs = gain / (loss + 1e-10)
        df[f'rsi_{period}'] = 100 - (100 / (1 + rs))
    
    df['rsi_oversold'] = (df['rsi_14'] < 30).astype(int)
    df['rsi_bullish'] = ((df['rsi_14'] > 50) & (df['rsi_14'] < 70)).astype(int)
    
    # MACD
    ema12 = df['close'].ewm(span=12, adjust=False).mean()
    ema26 = df['close'].ewm(span=26, adjust=False).mean()
    df['macd'] = ema12 - ema26
    df['macd_signal'] = df['macd'].ewm(span=9, adjust=False).mean()
    df['macd_hist'] = df['macd'] - df['macd_signal']
    df['macd_cross'] = (df['macd'] > df['macd_signal']).astype(int)
    df['macd_bullish'] = ((df['macd'] > df['macd_signal']) & (df['macd_hist'] > 0)).astype(int)
    
    # Stochastic
    for period in [14, 21]:
        low_min = df['low'].rolling(period).min()
        high_max = df['high'].rolling(period).max()
        df[f'stoch_k_{period}'] = 100 * (df['close'] - low_min) / (high_max - low_min + 1e-10)
        df[f'stoch_d_{period}'] = df[f'stoch_k_{period}'].rolling(3).mean()
    
    df['stoch_oversold'] = (df['stoch_k_14'] < 20).astype(int)
    df['stoch_bullish'] = ((df['stoch_k_14'] > df['stoch_d_14']) & (df['stoch_k_14'] < 80)).astype(int)
    
    # ATR
    high_low = df['high'] - df['low']
    high_close = abs(df['high'] - df['close'].shift())
    low_close = abs(df['low'] - df['close'].shift())
    tr = pd.concat([high_low, high_close, low_close], axis=1).max(axis=1)
    df['atr_14'] = tr.rolling(14).mean()
    df['atr_pips'] = df['atr_14'] * 10000
    
    # Bollinger Bands
    df['bb_middle'] = df['close'].rolling(20).mean()
    bb_std = df['close'].rolling(20).std()
    df['bb_upper'] = df['bb_middle'] + 2 * bb_std
    df['bb_lower'] = df['bb_middle'] - 2 * bb_std
    df['bb_width'] = (df['bb_upper'] - df['bb_lower']) / df['bb_middle'] * 100
    df['bb_position'] = (df['close'] - df['bb_lower']) / (df['bb_upper'] - df['bb_lower'] + 1e-10)
    df['bb_squeeze'] = (df['bb_width'] < df['bb_width'].rolling(50).mean()).astype(int)
    
    # Candle Patterns
    df['candle_body'] = df['close'] - df['open']
    df['candle_range'] = df['high'] - df['low']
    df['candle_body_pct'] = df['candle_body'] / (df['candle_range'] + 1e-10)
    df['upper_shadow'] = df['high'] - df[['open', 'close']].max(axis=1)
    df['lower_shadow'] = df[['open', 'close']].min(axis=1) - df['low']
    df['is_bullish'] = (df['close'] > df['open']).astype(int)
    df['is_hammer'] = ((df['lower_shadow'] > df['candle_body'].abs() * 2) & 
                       (df['upper_shadow'] < df['candle_body'].abs() * 0.5)).astype(int)
    df['bullish_engulfing'] = ((df['is_bullish'] == 1) & 
                               (df['is_bullish'].shift(1) == 0) &
                               (df['close'] > df['open'].shift(1))).astype(int)
    
    # Pivot Points
    df['pivot'] = (df['high'].shift() + df['low'].shift() + df['close'].shift()) / 3
    df['r1'] = 2 * df['pivot'] - df['low'].shift()
    df['s1'] = 2 * df['pivot'] - df['high'].shift()
    df['near_support'] = (df['close'] < df['s1'] * 1.001).astype(int)
    
    # Trend Strength
    df['trend_short'] = np.where(df['ema_10'] > df['ema_20'], 1, -1)
    df['trend_medium'] = np.where(df['ema_20'] > df['ema_50'], 1, -1)
    df['trend_long'] = np.where(df['ema_50'] > df['ema_200'], 1, -1)
    df['trend_alignment'] = df['trend_short'] + df['trend_medium'] + df['trend_long']
    df['strong_uptrend'] = (df['trend_alignment'] == 3).astype(int)
    
    # ==================== V5 ENHANCED FEATURES ====================
    # Momentum Consistency (NEW)
    df['mom_5'] = df['close'].pct_change(5) * 100
    df['mom_10'] = df['close'].pct_change(10) * 100
    df['mom_20'] = df['close'].pct_change(20) * 100
    df['mom_consistency'] = ((df['mom_5'] > 0) & (df['mom_10'] > 0) & (df['mom_20'] > 0)).astype(int)
    
    # RSI Momentum (NEW)
    df['rsi_momentum'] = df['rsi_14'] - df['rsi_14'].shift(5)
    df['rsi_rising'] = (df['rsi_momentum'] > 0).astype(int)
    
    # Volatility Regime (NEW)
    df['atr_ratio'] = df['atr_14'] / df['atr_14'].rolling(50).mean()
    df['high_volatility'] = (df['atr_ratio'] > 1.2).astype(int)
    df['low_volatility'] = (df['atr_ratio'] < 0.8).astype(int)
    
    # Combined Buy Score (V2 style)
    df['buy_score'] = (
        df['macd_bullish'] +
        df['rsi_bullish'] +
        df['sma_5_20_cross'] +
        df['golden_cross'] +
        df['price_above_all_ma'] +
        df['strong_uptrend'] +
        df['stoch_bullish'] +
        df['mom_consistency']
    )
    
    return df

print("Adding V5 features...")
train_df = add_features_v5(train_df)
test_df = add_features_v5(test_df)
print(f"✓ Features added: {len(train_df.columns)} columns")

Adding V5 features...
✓ Features added: 83 columns
✓ Features added: 83 columns


## 3. BUY Signal Labels (Same as V2)

In [19]:
def create_buy_labels(df, forward_periods=60, tp_pips=20, sl_pips=10):
    """
    Create BUY-only labels (Same logic as V2).
    BUY (1): TP reached before SL within forward_periods
    NOT_BUY (0): SL hit first or neither hit
    """
    df = df.copy()
    tp_move = tp_pips * 0.0001
    sl_move = sl_pips * 0.0001
    
    print(f"Creating BUY labels (TP={tp_pips} pips, SL={sl_pips} pips)...")
    
    # Calculate TP and SL prices
    df['tp_price'] = df['close'] + tp_move
    df['sl_price'] = df['close'] - sl_move
    
    # Future max/min within forward_periods
    df['future_max'] = df['high'].rolling(window=forward_periods).max().shift(-forward_periods)
    df['future_min'] = df['low'].rolling(window=forward_periods).min().shift(-forward_periods)
    
    # BUY = TP reachable AND SL not hit
    df['tp_reachable'] = df['future_max'] >= df['tp_price']
    df['sl_hit'] = df['future_min'] <= df['sl_price']
    df['is_buy'] = ((df['tp_reachable']) & (~df['sl_hit'])).astype(int)
    
    # Cleanup
    df.drop(['tp_price', 'sl_price', 'future_max', 'future_min', 'tp_reachable', 'sl_hit'], axis=1, inplace=True)
    
    return df

# Create labels with V2's R:R (2:1)
train_df = create_buy_labels(train_df, forward_periods=60, tp_pips=20, sl_pips=10)
test_df = create_buy_labels(test_df, forward_periods=60, tp_pips=20, sl_pips=10)

# Label distribution
print(f"\n📊 Label Distribution:")
print(f"Train - BUY: {train_df['is_buy'].mean()*100:.1f}%")
print(f"Test  - BUY: {test_df['is_buy'].mean()*100:.1f}%")

Creating BUY labels (TP=20 pips, SL=10 pips)...
Creating BUY labels (TP=20 pips, SL=10 pips)...

📊 Label Distribution:
Train - BUY: 5.2%
Test  - BUY: 7.3%
Creating BUY labels (TP=20 pips, SL=10 pips)...

📊 Label Distribution:
Train - BUY: 5.2%
Test  - BUY: 7.3%


## 4. Prepare Training Data

In [20]:
# Select features
exclude_cols = ['time', 'is_buy', 'open', 'high', 'low', 'close', 'volume', 'tick_volume']
feature_cols = [col for col in train_df.columns if col not in exclude_cols]
print(f"Features: {len(feature_cols)}")

# Drop NaN rows
train_clean = train_df.dropna(subset=feature_cols + ['is_buy']).copy()
test_clean = test_df.dropna(subset=feature_cols + ['is_buy']).copy()

print(f"\nTrain samples: {len(train_clean):,}")
print(f"Test samples: {len(test_clean):,}")

# Prepare X, y
X_train = train_clean[feature_cols].values
y_train = train_clean['is_buy'].values
X_test = test_clean[feature_cols].values
y_test = test_clean['is_buy'].values

# Scale features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

print(f"\n✓ Data ready")
print(f"  Class balance (train): {y_train.mean()*100:.1f}% BUY")
print(f"  Class balance (test): {y_test.mean()*100:.1f}% BUY")

Features: 77

Train samples: 1,859,293
Test samples: 296,579

Train samples: 1,859,293
Test samples: 296,579

✓ Data ready
  Class balance (train): 5.2% BUY
  Class balance (test): 7.3% BUY

✓ Data ready
  Class balance (train): 5.2% BUY
  Class balance (test): 7.3% BUY


## 5. Train Models with Calibration (V5 Key Innovation)

In [21]:
print("="*60)
print("🎯 TRAINING V5 MODELS (Direct - No Calibration)")
print("="*60)

# Class weight
neg_count = (y_train == 0).sum()
pos_count = (y_train == 1).sum()
scale_pos_weight = neg_count / pos_count
print(f"\nClass weight: {scale_pos_weight:.2f}")

# ==================== Model 1: XGBoost ====================
print("\nTraining XGBoost...")
xgb_model = xgb.XGBClassifier(
    n_estimators=500,
    max_depth=10,
    learning_rate=0.03,
    subsample=0.8,
    colsample_bytree=0.8,
    scale_pos_weight=scale_pos_weight,
    random_state=42,
    eval_metric='logloss',
    verbosity=0,
    tree_method='hist',
    device='cuda' if GPU_AVAILABLE else 'cpu'
)
xgb_model.fit(X_train_scaled, y_train)
xgb_proba = xgb_model.predict_proba(X_test_scaled)
xgb_pred = xgb_model.predict(X_test_scaled)
xgb_acc = accuracy_score(y_test, xgb_pred)
xgb_prec = precision_score(y_test, xgb_pred, zero_division=0)
print(f"  ✓ XGBoost - Accuracy: {xgb_acc*100:.2f}%, Precision: {xgb_prec*100:.2f}%")

# ==================== Model 2: LightGBM ====================
print("\nTraining LightGBM...")
lgb_model = lgb.LGBMClassifier(
    n_estimators=500,
    max_depth=10,
    learning_rate=0.03,
    subsample=0.8,
    colsample_bytree=0.8,
    scale_pos_weight=scale_pos_weight,
    random_state=42,
    verbose=-1,
    n_jobs=-1
)
lgb_model.fit(X_train_scaled, y_train)
lgb_proba = lgb_model.predict_proba(X_test_scaled)
lgb_pred = lgb_model.predict(X_test_scaled)
lgb_acc = accuracy_score(y_test, lgb_pred)
lgb_prec = precision_score(y_test, lgb_pred, zero_division=0)
print(f"  ✓ LightGBM - Accuracy: {lgb_acc*100:.2f}%, Precision: {lgb_prec*100:.2f}%")

# ==================== Model 3: Random Forest ====================
print("\nTraining Random Forest...")
rf_model = RandomForestClassifier(
    n_estimators=300,
    max_depth=15,
    min_samples_split=10,
    class_weight='balanced',
    random_state=42,
    n_jobs=-1
)
rf_model.fit(X_train_scaled, y_train)
rf_proba = rf_model.predict_proba(X_test_scaled)
rf_pred = rf_model.predict(X_test_scaled)
rf_acc = accuracy_score(y_test, rf_pred)
rf_prec = precision_score(y_test, rf_pred, zero_division=0)
print(f"  ✓ Random Forest - Accuracy: {rf_acc*100:.2f}%, Precision: {rf_prec*100:.2f}%")

print(f"\n{'='*60}")
print(f"📊 MODEL COMPARISON:")
print(f"   XGBoost:       {xgb_acc*100:.2f}% acc, {xgb_prec*100:.2f}% precision")
print(f"   LightGBM:      {lgb_acc*100:.2f}% acc, {lgb_prec*100:.2f}% precision")
print(f"   Random Forest: {rf_acc*100:.2f}% acc, {rf_prec*100:.2f}% precision")

🎯 TRAINING V5 MODELS (Direct - No Calibration)

Class weight: 18.11

Training XGBoost...
  ✓ XGBoost - Accuracy: 74.67%, Precision: 16.33%

Training LightGBM...
  ✓ XGBoost - Accuracy: 74.67%, Precision: 16.33%

Training LightGBM...
  ✓ LightGBM - Accuracy: 63.93%, Precision: 14.48%

Training Random Forest...
  ✓ LightGBM - Accuracy: 63.93%, Precision: 14.48%

Training Random Forest...
  ✓ Random Forest - Accuracy: 73.37%, Precision: 16.66%

📊 MODEL COMPARISON:
   XGBoost:       74.67% acc, 16.33% precision
   LightGBM:      63.93% acc, 14.48% precision
   Random Forest: 73.37% acc, 16.66% precision
  ✓ Random Forest - Accuracy: 73.37%, Precision: 16.66%

📊 MODEL COMPARISON:
   XGBoost:       74.67% acc, 16.33% precision
   LightGBM:      63.93% acc, 14.48% precision
   Random Forest: 73.37% acc, 16.66% precision


## 6. Ensemble with Confidence Analysis

In [22]:
print("="*60)
print("🔍 V5 ENSEMBLE ANALYSIS")
print("="*60)

# Weighted Ensemble (XGB slightly higher weight)
weights = [0.4, 0.35, 0.25]  # XGB, LGB, RF
avg_proba = (weights[0] * xgb_proba + weights[1] * lgb_proba + weights[2] * rf_proba)
buy_prob = avg_proba[:, 1] * 100

# Model Agreement Bonus
all_agree_buy = (xgb_pred == 1) & (lgb_pred == 1) & (rf_pred == 1)
confidence = buy_prob.copy()
confidence[all_agree_buy] = np.minimum(confidence[all_agree_buy] + 5, 100)

print(f"\nModel Agreement on BUY: {all_agree_buy.sum():,} signals")

# ==================== ACCURACY BY CONFIDENCE ====================
print(f"\n📊 BUY Signal Accuracy by Confidence Level:")
print("-"*60)
print(f"{'Confidence':>12} | {'Signals':>8} | {'Accuracy':>10} | {'Precision':>10}")
print("-"*60)

results = []
for min_conf in [50, 60, 70, 75, 80, 85, 90]:
    mask = confidence >= min_conf
    if mask.sum() > 0:
        signals = mask.sum()
        correct = y_test[mask].sum()
        accuracy = correct / signals * 100
        results.append({'conf': min_conf, 'signals': signals, 'acc': accuracy})
        print(f"{min_conf:>10}%+ | {signals:>8} | {accuracy:>9.1f}% | {accuracy:>9.1f}%")

# Find optimal threshold
print(f"\n💡 Recommendations:")
for r in results:
    if r['acc'] >= 80 and r['signals'] >= 10:
        print(f"   ✓ Use {r['conf']}% threshold: {r['signals']} signals, {r['acc']:.1f}% accuracy")

🔍 V5 ENSEMBLE ANALYSIS

Model Agreement on BUY: 67,659 signals

📊 BUY Signal Accuracy by Confidence Level:
------------------------------------------------------------
  Confidence |  Signals |   Accuracy |  Precision
------------------------------------------------------------
        50%+ |    94250 |      16.1% |      16.1%
        60%+ |    70528 |      17.7% |      17.7%
        70%+ |    49262 |      20.0% |      20.0%
        75%+ |    35835 |      21.1% |      21.1%
        80%+ |    20430 |      22.2% |      22.2%
        85%+ |     6914 |      23.2% |      23.2%
        90%+ |      405 |      25.7% |      25.7%

💡 Recommendations:


## 7. Compare with V2 Baseline

In [23]:
print("="*60)
print("📊 V5 vs V2 COMPARISON")
print("="*60)

# V2 uses BUY vs SELL approach, so we need to recreate the same test data
# Load fresh test data for V2
test_df_v2 = pd.read_csv(DATA_DIR / 'EUR_USD_test.csv')

# V2 Feature Engineering (simplified version of add_technical_indicators)
def add_v2_features(df):
    """V2 compatible features"""
    df = df.copy()
    
    # Moving Averages
    for period in [5, 10, 20, 50, 100, 200]:
        df[f'sma_{period}'] = df['close'].rolling(period).mean()
        df[f'ema_{period}'] = df['close'].ewm(span=period, adjust=False).mean()
    
    # MA Crossovers
    df['sma_5_20_cross'] = (df['sma_5'] > df['sma_20']).astype(int)
    df['sma_20_50_cross'] = (df['sma_20'] > df['sma_50']).astype(int)
    df['ema_10_50_cross'] = (df['ema_10'] > df['ema_50']).astype(int)
    df['golden_cross'] = (df['sma_50'] > df['sma_200']).astype(int)
    
    df['price_vs_sma20'] = (df['close'] - df['sma_20']) / df['sma_20'] * 100
    df['price_vs_sma50'] = (df['close'] - df['sma_50']) / df['sma_50'] * 100
    df['price_vs_ema20'] = (df['close'] - df['ema_20']) / df['ema_20'] * 100
    df['price_above_all_ma'] = ((df['close'] > df['sma_20']) & 
                                 (df['close'] > df['sma_50']) & 
                                 (df['close'] > df['ema_20'])).astype(int)
    
    # RSI
    for period in [7, 14, 21]:
        delta = df['close'].diff()
        gain = delta.where(delta > 0, 0).rolling(period).mean()
        loss = (-delta.where(delta < 0, 0)).rolling(period).mean()
        rs = gain / (loss + 1e-10)
        df[f'rsi_{period}'] = 100 - (100 / (1 + rs))
    
    df['rsi_oversold'] = (df['rsi_14'] < 30).astype(int)
    df['rsi_bullish'] = ((df['rsi_14'] > 50) & (df['rsi_14'] < 70)).astype(int)
    
    # MACD
    ema12 = df['close'].ewm(span=12, adjust=False).mean()
    ema26 = df['close'].ewm(span=26, adjust=False).mean()
    df['macd'] = ema12 - ema26
    df['macd_signal'] = df['macd'].ewm(span=9, adjust=False).mean()
    df['macd_hist'] = df['macd'] - df['macd_signal']
    df['macd_cross'] = (df['macd'] > df['macd_signal']).astype(int)
    df['macd_bullish'] = ((df['macd'] > df['macd_signal']) & (df['macd_hist'] > 0)).astype(int)
    
    # Stochastic
    for period in [14, 21]:
        low_min = df['low'].rolling(period).min()
        high_max = df['high'].rolling(period).max()
        df[f'stoch_k_{period}'] = 100 * (df['close'] - low_min) / (high_max - low_min + 1e-10)
        df[f'stoch_d_{period}'] = df[f'stoch_k_{period}'].rolling(3).mean()
    
    df['stoch_oversold'] = (df['stoch_k_14'] < 20).astype(int)
    df['stoch_bullish'] = ((df['stoch_k_14'] > df['stoch_d_14']) & (df['stoch_k_14'] < 80)).astype(int)
    
    # ATR
    high_low = df['high'] - df['low']
    high_close = abs(df['high'] - df['close'].shift())
    low_close = abs(df['low'] - df['close'].shift())
    tr = pd.concat([high_low, high_close, low_close], axis=1).max(axis=1)
    df['atr_14'] = tr.rolling(14).mean()
    df['atr_pips'] = df['atr_14'] * 10000
    
    # Bollinger
    df['bb_middle'] = df['close'].rolling(20).mean()
    bb_std = df['close'].rolling(20).std()
    df['bb_upper'] = df['bb_middle'] + 2 * bb_std
    df['bb_lower'] = df['bb_middle'] - 2 * bb_std
    df['bb_width'] = (df['bb_upper'] - df['bb_lower']) / df['bb_middle'] * 100
    df['bb_position'] = (df['close'] - df['bb_lower']) / (df['bb_upper'] - df['bb_lower'] + 1e-10)
    df['bb_squeeze'] = (df['bb_width'] < df['bb_width'].rolling(50).mean()).astype(int)
    
    # Candle
    df['candle_body'] = df['close'] - df['open']
    df['candle_range'] = df['high'] - df['low']
    df['candle_body_pct'] = df['candle_body'] / (df['candle_range'] + 1e-10)
    df['upper_shadow'] = df['high'] - df[['open', 'close']].max(axis=1)
    df['lower_shadow'] = df[['open', 'close']].min(axis=1) - df['low']
    df['is_bullish'] = (df['close'] > df['open']).astype(int)
    df['is_hammer'] = ((df['lower_shadow'] > df['candle_body'].abs() * 2) & 
                       (df['upper_shadow'] < df['candle_body'].abs() * 0.5)).astype(int)
    df['bullish_engulfing'] = ((df['is_bullish'] == 1) & 
                               (df['is_bullish'].shift(1) == 0) &
                               (df['close'] > df['open'].shift(1))).astype(int)
    
    # Pivot
    df['pivot'] = (df['high'].shift() + df['low'].shift() + df['close'].shift()) / 3
    df['r1'] = 2 * df['pivot'] - df['low'].shift()
    df['s1'] = 2 * df['pivot'] - df['high'].shift()
    df['r2'] = df['pivot'] + (df['high'].shift() - df['low'].shift())
    df['s2'] = df['pivot'] - (df['high'].shift() - df['low'].shift())
    df['near_support'] = (df['close'] < df['s1'] * 1.001).astype(int)
    
    # Trend
    df['trend_short'] = np.where(df['ema_10'] > df['ema_20'], 1, -1)
    df['trend_medium'] = np.where(df['ema_20'] > df['ema_50'], 1, -1)
    df['trend_long'] = np.where(df['ema_50'] > df['ema_200'], 1, -1)
    df['trend_alignment'] = df['trend_short'] + df['trend_medium'] + df['trend_long']
    df['strong_uptrend'] = (df['trend_alignment'] == 3).astype(int)
    
    df['buy_score'] = (
        df['macd_bullish'] + df['rsi_bullish'] + df['sma_5_20_cross'] +
        df['golden_cross'] + df['price_above_all_ma'] + df['strong_uptrend']
    )
    
    return df

# V2 Signal Labels (BUY vs SELL)
def create_v2_signal_labels(df, forward_periods=60, min_move_pips=15):
    df = df.copy()
    min_move = min_move_pips * 0.0001
    
    df['future_max'] = df['high'].rolling(window=forward_periods).max().shift(-forward_periods)
    df['future_min'] = df['low'].rolling(window=forward_periods).min().shift(-forward_periods)
    
    df['up_move'] = df['future_max'] - df['close']
    df['down_move'] = df['close'] - df['future_min']
    
    conditions = [
        (df['up_move'] >= min_move) & (df['up_move'] > df['down_move'] * 1.5),
        (df['down_move'] >= min_move) & (df['down_move'] > df['up_move'] * 1.5),
    ]
    choices = [1, -1]
    df['signal'] = np.select(conditions, choices, default=0)
    
    df.drop(['future_max', 'future_min', 'up_move', 'down_move'], axis=1, inplace=True)
    return df

# Prepare V2 test data
test_df_v2 = add_v2_features(test_df_v2)
test_df_v2 = create_v2_signal_labels(test_df_v2)

# Filter BUY/SELL only
test_binary_v2 = test_df_v2[test_df_v2['signal'] != 0].copy()

# Load V2 models
v2_dir = BASE_DIR / 'models' / 'signal_generator_v2'
try:
    v2_xgb = joblib.load(v2_dir / 'xgboost_v2.joblib')
    v2_lgb = joblib.load(v2_dir / 'lightgbm_v2.joblib')
    v2_rf = joblib.load(v2_dir / 'rf_v2.joblib')
    v2_scaler = joblib.load(v2_dir / 'scaler_v2.joblib')
    v2_feature_cols = joblib.load(v2_dir / 'feature_cols_v2.joblib')
    
    # Check missing features
    missing = [c for c in v2_feature_cols if c not in test_binary_v2.columns]
    if missing:
        print(f"Missing features for V2: {missing[:5]}...")
        # Add missing with 0
        for c in missing:
            test_binary_v2[c] = 0
    
    test_binary_v2_clean = test_binary_v2.dropna(subset=v2_feature_cols).copy()
    X_test_v2 = test_binary_v2_clean[v2_feature_cols].values
    y_test_v2 = ((test_binary_v2_clean['signal'].values + 1) / 2).astype(int)
    
    X_test_v2_scaled = v2_scaler.transform(X_test_v2)
    
    # V2 predictions
    v2_xgb_proba = v2_xgb.predict_proba(X_test_v2_scaled)
    v2_lgb_proba = v2_lgb.predict_proba(X_test_v2_scaled)
    v2_rf_proba = v2_rf.predict_proba(X_test_v2_scaled)
    
    v2_xgb_pred = (v2_xgb_proba[:, 1] > 0.5).astype(int)
    v2_lgb_pred = (v2_lgb_proba[:, 1] > 0.5).astype(int)
    v2_rf_pred = (v2_rf_proba[:, 1] > 0.5).astype(int)
    
    v2_avg_proba = (0.4 * v2_xgb_proba + 0.35 * v2_lgb_proba + 0.25 * v2_rf_proba)
    v2_buy_prob = v2_avg_proba[:, 1] * 100
    
    # Model agreement bonus
    v2_all_agree = (v2_xgb_pred == 1) & (v2_lgb_pred == 1) & (v2_rf_pred == 1)
    v2_confidence = v2_buy_prob.copy()
    v2_confidence[v2_all_agree] = np.minimum(v2_confidence[v2_all_agree] + 5, 100)
    
    print(f"\n✅ V2 Test Data: {len(test_binary_v2_clean):,} samples (BUY/SELL only)")
    print(f"   BUY ratio: {y_test_v2.mean()*100:.1f}%")
    
    # V2 Accuracy by Confidence
    print(f"\n📊 V2 BUY Signal Accuracy:")
    print("-"*50)
    for thresh in [70, 75, 80, 85, 90]:
        mask = (v2_confidence >= thresh)
        if mask.sum() > 0:
            signals = mask.sum()
            acc = y_test_v2[mask].mean() * 100
            print(f"   Conf >= {thresh}%: {signals:>5} signals | Accuracy: {acc:.1f}%")
    
    print(f"\n" + "="*60)
    print("📊 V5 vs V2 COMPARISON (Same labeling approach)")
    print("="*60)
    
    # For fair comparison, we need V5 on the same BUY/SELL data
    # But V5 was trained on different labels. Let's just show both results.
    print("\n⚠️  Note: V5 uses different labeling (BUY-only with TP/SL)")
    print("         V2 uses BUY vs SELL classification")
    print("\n📊 V5 Results (BUY-only TP=20/SL=10):")
    for thresh in [70, 75, 80, 85, 90]:
        mask = confidence >= thresh
        if mask.sum() > 0:
            signals = mask.sum()
            acc = y_test[mask].mean() * 100
            print(f"   Conf >= {thresh}%: {signals:>5} signals | Accuracy: {acc:.1f}%")
            
except Exception as e:
    import traceback
    print(f"Error: {e}")
    traceback.print_exc()

📊 V5 vs V2 COMPARISON
Missing features for V2: ['roc_5', 'roc_10', 'roc_20', 'momentum_10', 'momentum_20']...
Missing features for V2: ['roc_5', 'roc_10', 'roc_20', 'momentum_10', 'momentum_20']...

✅ V2 Test Data: 80,296 samples (BUY/SELL only)
   BUY ratio: 52.2%

📊 V2 BUY Signal Accuracy:
--------------------------------------------------
   Conf >= 70%:  2344 signals | Accuracy: 51.8%
   Conf >= 75%:   663 signals | Accuracy: 55.1%
   Conf >= 80%:   112 signals | Accuracy: 58.9%
   Conf >= 85%:     3 signals | Accuracy: 33.3%

📊 V5 vs V2 COMPARISON (Same labeling approach)

⚠️  Note: V5 uses different labeling (BUY-only with TP/SL)
         V2 uses BUY vs SELL classification

📊 V5 Results (BUY-only TP=20/SL=10):
   Conf >= 70%: 49262 signals | Accuracy: 20.0%
   Conf >= 75%: 35835 signals | Accuracy: 21.1%
   Conf >= 80%: 20430 signals | Accuracy: 22.2%
   Conf >= 85%:  6914 signals | Accuracy: 23.2%
   Conf >= 90%:   405 signals | Accuracy: 25.7%

✅ V2 Test Data: 80,296 samples (B

## 8. Backtest V5

In [24]:
def backtest_v5(df, probabilities, min_confidence=80, tp_pips=20, sl_pips=10):
    """
    Backtest BUY signals with fixed SL/TP (same as V2 labeling).
    """
    results = []
    tp_move = tp_pips * 0.0001
    sl_move = sl_pips * 0.0001
    
    # BUY signals
    buy_indices = np.where(probabilities >= min_confidence)[0]
    
    print(f"Backtesting {len(buy_indices)} BUY signals (Conf >= {min_confidence}%)...")
    
    for idx in buy_indices:
        if idx + 60 >= len(df):
            continue
            
        entry = df['close'].iloc[idx]
        tp_price = entry + tp_move
        sl_price = entry - sl_move
        
        # Check outcome
        future = df.iloc[idx+1 : idx+61]
        result = 'TIMEOUT'
        pnl = 0
        
        for _, row in future.iterrows():
            if row['low'] <= sl_price:
                result = 'LOSS'
                pnl = -sl_pips
                break
            if row['high'] >= tp_price:
                result = 'WIN'
                pnl = tp_pips
                break
        
        if result == 'TIMEOUT':
            end_price = future['close'].iloc[-1] if len(future) > 0 else entry
            pnl = (end_price - entry) * 10000
            
        results.append({'pnl': pnl, 'result': result})
    
    return results

# Run Backtest
print("="*60)
print("📈 V5 BACKTEST RESULTS")
print("="*60)

for conf in [70, 75, 80, 85, 90]:
    bt_results = backtest_v5(test_clean, confidence, min_confidence=conf)
    
    if len(bt_results) > 0:
        wins = len([r for r in bt_results if r['pnl'] > 0])
        total = len(bt_results)
        wr = wins / total * 100
        total_pips = sum([r['pnl'] for r in bt_results])
        
        # Profit Factor
        gross_profit = sum([r['pnl'] for r in bt_results if r['pnl'] > 0])
        gross_loss = abs(sum([r['pnl'] for r in bt_results if r['pnl'] < 0]))
        pf = gross_profit / gross_loss if gross_loss > 0 else float('inf')
        
        print(f"Conf >= {conf}%: {total:>4} signals | WR: {wr:>5.1f}% | Pips: {total_pips:>7.1f} | PF: {pf:.2f}")

📈 V5 BACKTEST RESULTS
Backtesting 49262 BUY signals (Conf >= 70%)...
Conf >= 70%: 49262 signals | WR:  41.3% | Pips: 22211.7 | PF: 1.08
Backtesting 35835 BUY signals (Conf >= 75%)...
Conf >= 70%: 49262 signals | WR:  41.3% | Pips: 22211.7 | PF: 1.08
Backtesting 35835 BUY signals (Conf >= 75%)...
Conf >= 75%: 35835 signals | WR:  40.7% | Pips: 15695.0 | PF: 1.08
Backtesting 20430 BUY signals (Conf >= 80%)...
Conf >= 75%: 35835 signals | WR:  40.7% | Pips: 15695.0 | PF: 1.08
Backtesting 20430 BUY signals (Conf >= 80%)...
Conf >= 80%: 20430 signals | WR:  39.3% | Pips:  5664.7 | PF: 1.05
Backtesting 6914 BUY signals (Conf >= 85%)...
Conf >= 80%: 20430 signals | WR:  39.3% | Pips:  5664.7 | PF: 1.05
Backtesting 6914 BUY signals (Conf >= 85%)...
Conf >= 85%: 6914 signals | WR:  37.8% | Pips:   823.3 | PF: 1.02
Backtesting 405 BUY signals (Conf >= 90%)...
Conf >= 85%: 6914 signals | WR:  37.8% | Pips:   823.3 | PF: 1.02
Backtesting 405 BUY signals (Conf >= 90%)...
Conf >= 90%:  405 signals |

## 9. V5 Improved: BUY vs SELL Approach (Same as V2)

In [25]:
# ================================================================
# V5 IMPROVED: Use V2's BUY vs SELL approach + Time Features
# ================================================================
print("="*70)
print("🚀 V5 IMPROVED: BUY vs SELL + Time Features")
print("="*70)

# Reload fresh data
train_df_v5 = pd.read_csv(DATA_DIR / 'EUR_USD_1min.csv')
test_df_v5 = pd.read_csv(DATA_DIR / 'EUR_USD_test.csv')

# Rename timestamp if needed
if 'timestamp' in train_df_v5.columns:
    train_df_v5.rename(columns={'timestamp': 'time'}, inplace=True)
if 'timestamp' in test_df_v5.columns:
    test_df_v5.rename(columns={'timestamp': 'time'}, inplace=True)

train_df_v5['time'] = pd.to_datetime(train_df_v5['time'])
test_df_v5['time'] = pd.to_datetime(test_df_v5['time'])

def add_v5_improved_features(df):
    """V2 Core Features + V5 Time/Momentum Features"""
    df = df.copy()
    
    # ==================== V5 TIME FEATURES ====================
    df['hour'] = df['time'].dt.hour
    df['day_of_week'] = df['time'].dt.dayofweek
    df['is_asian'] = ((df['hour'] >= 0) & (df['hour'] < 8)).astype(int)
    df['is_london'] = ((df['hour'] >= 8) & (df['hour'] < 16)).astype(int)
    df['is_ny'] = ((df['hour'] >= 13) & (df['hour'] < 21)).astype(int)
    df['is_overlap'] = ((df['hour'] >= 13) & (df['hour'] < 16)).astype(int)
    df['is_low_liquidity'] = ((df['hour'] >= 21) | (df['hour'] < 2)).astype(int)
    
    # ==================== V2 CORE FEATURES ====================
    # Moving Averages
    for period in [5, 10, 20, 50, 100, 200]:
        df[f'sma_{period}'] = df['close'].rolling(period).mean()
        df[f'ema_{period}'] = df['close'].ewm(span=period, adjust=False).mean()
    
    # MA Crossovers
    df['sma_5_20_cross'] = (df['sma_5'] > df['sma_20']).astype(int)
    df['sma_20_50_cross'] = (df['sma_20'] > df['sma_50']).astype(int)
    df['ema_10_50_cross'] = (df['ema_10'] > df['ema_50']).astype(int)
    df['golden_cross'] = (df['sma_50'] > df['sma_200']).astype(int)
    
    df['price_vs_sma20'] = (df['close'] - df['sma_20']) / df['sma_20'] * 100
    df['price_vs_sma50'] = (df['close'] - df['sma_50']) / df['sma_50'] * 100
    df['price_vs_ema20'] = (df['close'] - df['ema_20']) / df['ema_20'] * 100
    df['price_above_all_ma'] = ((df['close'] > df['sma_20']) & 
                                 (df['close'] > df['sma_50']) & 
                                 (df['close'] > df['ema_20'])).astype(int)
    
    # RSI
    for period in [7, 14, 21]:
        delta = df['close'].diff()
        gain = delta.where(delta > 0, 0).rolling(period).mean()
        loss = (-delta.where(delta < 0, 0)).rolling(period).mean()
        rs = gain / (loss + 1e-10)
        df[f'rsi_{period}'] = 100 - (100 / (1 + rs))
    
    df['rsi_oversold'] = (df['rsi_14'] < 30).astype(int)
    df['rsi_bullish'] = ((df['rsi_14'] > 50) & (df['rsi_14'] < 70)).astype(int)
    
    # MACD
    ema12 = df['close'].ewm(span=12, adjust=False).mean()
    ema26 = df['close'].ewm(span=26, adjust=False).mean()
    df['macd'] = ema12 - ema26
    df['macd_signal'] = df['macd'].ewm(span=9, adjust=False).mean()
    df['macd_hist'] = df['macd'] - df['macd_signal']
    df['macd_cross'] = (df['macd'] > df['macd_signal']).astype(int)
    df['macd_bullish'] = ((df['macd'] > df['macd_signal']) & (df['macd_hist'] > 0)).astype(int)
    
    # Stochastic
    for period in [14, 21]:
        low_min = df['low'].rolling(period).min()
        high_max = df['high'].rolling(period).max()
        df[f'stoch_k_{period}'] = 100 * (df['close'] - low_min) / (high_max - low_min + 1e-10)
        df[f'stoch_d_{period}'] = df[f'stoch_k_{period}'].rolling(3).mean()
    
    df['stoch_oversold'] = (df['stoch_k_14'] < 20).astype(int)
    df['stoch_bullish'] = ((df['stoch_k_14'] > df['stoch_d_14']) & (df['stoch_k_14'] < 80)).astype(int)
    
    # ROC
    for period in [5, 10, 20]:
        df[f'roc_{period}'] = df['close'].pct_change(period) * 100
    
    # Momentum
    df['momentum_10'] = df['close'] - df['close'].shift(10)
    df['momentum_20'] = df['close'] - df['close'].shift(20)
    
    # ATR
    high_low = df['high'] - df['low']
    high_close = abs(df['high'] - df['close'].shift())
    low_close = abs(df['low'] - df['close'].shift())
    tr = pd.concat([high_low, high_close, low_close], axis=1).max(axis=1)
    df['atr_14'] = tr.rolling(14).mean()
    df['atr_20'] = tr.rolling(20).mean()
    df['atr_pips'] = df['atr_14'] * 10000
    df['atr_pct'] = df['atr_14'] / df['close'] * 100
    
    # Bollinger
    df['bb_middle'] = df['close'].rolling(20).mean()
    bb_std = df['close'].rolling(20).std()
    df['bb_upper'] = df['bb_middle'] + 2 * bb_std
    df['bb_lower'] = df['bb_middle'] - 2 * bb_std
    df['bb_width'] = (df['bb_upper'] - df['bb_lower']) / df['bb_middle'] * 100
    df['bb_position'] = (df['close'] - df['bb_lower']) / (df['bb_upper'] - df['bb_lower'] + 1e-10)
    df['bb_squeeze'] = (df['bb_width'] < df['bb_width'].rolling(50).mean()).astype(int)
    
    # Candles
    df['candle_body'] = df['close'] - df['open']
    df['candle_range'] = df['high'] - df['low']
    df['candle_body_pct'] = df['candle_body'] / (df['candle_range'] + 1e-10)
    df['upper_shadow'] = df['high'] - df[['open', 'close']].max(axis=1)
    df['lower_shadow'] = df[['open', 'close']].min(axis=1) - df['low']
    df['is_bullish'] = (df['close'] > df['open']).astype(int)
    df['is_hammer'] = ((df['lower_shadow'] > df['candle_body'].abs() * 2) & 
                       (df['upper_shadow'] < df['candle_body'].abs() * 0.5)).astype(int)
    df['bullish_engulfing'] = ((df['is_bullish'] == 1) & 
                               (df['is_bullish'].shift(1) == 0) &
                               (df['close'] > df['open'].shift(1))).astype(int)
    
    # Pivot
    df['pivot'] = (df['high'].shift() + df['low'].shift() + df['close'].shift()) / 3
    df['r1'] = 2 * df['pivot'] - df['low'].shift()
    df['s1'] = 2 * df['pivot'] - df['high'].shift()
    df['r2'] = df['pivot'] + (df['high'].shift() - df['low'].shift())
    df['s2'] = df['pivot'] - (df['high'].shift() - df['low'].shift())
    df['near_support'] = (df['close'] < df['s1'] * 1.001).astype(int)
    
    # Trend
    df['trend_short'] = np.where(df['ema_10'] > df['ema_20'], 1, -1)
    df['trend_medium'] = np.where(df['ema_20'] > df['ema_50'], 1, -1)
    df['trend_long'] = np.where(df['ema_50'] > df['ema_200'], 1, -1)
    df['trend_alignment'] = df['trend_short'] + df['trend_medium'] + df['trend_long']
    df['strong_uptrend'] = (df['trend_alignment'] == 3).astype(int)
    
    df['buy_score'] = (
        df['macd_bullish'] + df['rsi_bullish'] + df['sma_5_20_cross'] +
        df['golden_cross'] + df['price_above_all_ma'] + df['strong_uptrend']
    )
    
    return df

# V2 Signal Labels
def create_signal_labels_v5(df, forward_periods=60, min_move_pips=15):
    df = df.copy()
    min_move = min_move_pips * 0.0001
    
    df['future_max'] = df['high'].rolling(window=forward_periods).max().shift(-forward_periods)
    df['future_min'] = df['low'].rolling(window=forward_periods).min().shift(-forward_periods)
    
    df['up_move'] = df['future_max'] - df['close']
    df['down_move'] = df['close'] - df['future_min']
    
    conditions = [
        (df['up_move'] >= min_move) & (df['up_move'] > df['down_move'] * 1.5),
        (df['down_move'] >= min_move) & (df['down_move'] > df['up_move'] * 1.5),
    ]
    choices = [1, -1]
    df['signal'] = np.select(conditions, choices, default=0)
    
    df.drop(['future_max', 'future_min', 'up_move', 'down_move'], axis=1, inplace=True)
    return df

# Apply
print("Adding V5 improved features...")
train_df_v5 = add_v5_improved_features(train_df_v5)
test_df_v5 = add_v5_improved_features(test_df_v5)

train_df_v5 = create_signal_labels_v5(train_df_v5)
test_df_v5 = create_signal_labels_v5(test_df_v5)

# Filter BUY/SELL only
train_binary_v5 = train_df_v5[train_df_v5['signal'] != 0].copy()
test_binary_v5 = test_df_v5[test_df_v5['signal'] != 0].copy()

# Prepare data
exclude_v5 = ['time', 'signal', 'open', 'high', 'low', 'close', 'volume', 'tick_volume']
feature_cols_v5 = [c for c in train_binary_v5.columns if c not in exclude_v5]

train_clean_v5 = train_binary_v5.dropna(subset=feature_cols_v5).copy()
test_clean_v5 = test_binary_v5.dropna(subset=feature_cols_v5).copy()

X_train_v5 = train_clean_v5[feature_cols_v5].values
y_train_v5 = ((train_clean_v5['signal'].values + 1) / 2).astype(int)  # -1->0, 1->1

X_test_v5 = test_clean_v5[feature_cols_v5].values
y_test_v5 = ((test_clean_v5['signal'].values + 1) / 2).astype(int)

scaler_v5 = StandardScaler()
X_train_v5_scaled = scaler_v5.fit_transform(X_train_v5)
X_test_v5_scaled = scaler_v5.transform(X_test_v5)

print(f"\n✓ Train: {len(train_clean_v5):,} samples (BUY/SELL only)")
print(f"✓ Test: {len(test_clean_v5):,} samples")
print(f"✓ Features: {len(feature_cols_v5)}")
print(f"✓ BUY ratio (train): {y_train_v5.mean()*100:.1f}%")
print(f"✓ BUY ratio (test): {y_test_v5.mean()*100:.1f}%")

🚀 V5 IMPROVED: BUY vs SELL + Time Features
Adding V5 improved features...
Adding V5 improved features...

✓ Train: 393,249 samples (BUY/SELL only)
✓ Test: 80,296 samples
✓ Features: 78
✓ BUY ratio (train): 49.3%
✓ BUY ratio (test): 52.2%

✓ Train: 393,249 samples (BUY/SELL only)
✓ Test: 80,296 samples
✓ Features: 78
✓ BUY ratio (train): 49.3%
✓ BUY ratio (test): 52.2%


In [26]:
# ================================================================
# TRAIN V5 IMPROVED MODELS
# ================================================================
print("="*70)
print("🎯 TRAINING V5 IMPROVED MODELS")
print("="*70)

# Class weight
neg_v5 = (y_train_v5 == 0).sum()
pos_v5 = (y_train_v5 == 1).sum()
scale_weight_v5 = neg_v5 / pos_v5
print(f"Class weight: {scale_weight_v5:.2f}")

# XGBoost
print("\nTraining XGBoost V5...")
xgb_v5 = xgb.XGBClassifier(
    n_estimators=500,
    max_depth=10,
    learning_rate=0.03,
    subsample=0.8,
    colsample_bytree=0.8,
    random_state=42,
    eval_metric='logloss',
    verbosity=0,
    tree_method='hist',
    device='cuda' if GPU_AVAILABLE else 'cpu'
)
xgb_v5.fit(X_train_v5_scaled, y_train_v5)
xgb_v5_pred = xgb_v5.predict(X_test_v5_scaled)
xgb_v5_proba = xgb_v5.predict_proba(X_test_v5_scaled)
print(f"  ✓ XGBoost: {accuracy_score(y_test_v5, xgb_v5_pred)*100:.2f}%")

# LightGBM
print("\nTraining LightGBM V5...")
lgb_v5 = lgb.LGBMClassifier(
    n_estimators=500,
    max_depth=10,
    learning_rate=0.03,
    subsample=0.8,
    colsample_bytree=0.8,
    random_state=42,
    verbose=-1,
    n_jobs=-1
)
lgb_v5.fit(X_train_v5_scaled, y_train_v5)
lgb_v5_pred = lgb_v5.predict(X_test_v5_scaled)
lgb_v5_proba = lgb_v5.predict_proba(X_test_v5_scaled)
print(f"  ✓ LightGBM: {accuracy_score(y_test_v5, lgb_v5_pred)*100:.2f}%")

# Random Forest
print("\nTraining Random Forest V5...")
rf_v5 = RandomForestClassifier(
    n_estimators=300,
    max_depth=15,
    min_samples_split=10,
    class_weight='balanced',
    random_state=42,
    n_jobs=-1
)
rf_v5.fit(X_train_v5_scaled, y_train_v5)
rf_v5_pred = rf_v5.predict(X_test_v5_scaled)
rf_v5_proba = rf_v5.predict_proba(X_test_v5_scaled)
print(f"  ✓ Random Forest: {accuracy_score(y_test_v5, rf_v5_pred)*100:.2f}%")

🎯 TRAINING V5 IMPROVED MODELS
Class weight: 1.03

Training XGBoost V5...
  ✓ XGBoost: 50.22%

Training LightGBM V5...
  ✓ XGBoost: 50.22%

Training LightGBM V5...
  ✓ LightGBM: 49.67%

Training Random Forest V5...
  ✓ LightGBM: 49.67%

Training Random Forest V5...
  ✓ Random Forest: 51.34%
  ✓ Random Forest: 51.34%


In [27]:
# ================================================================
# V5 IMPROVED ENSEMBLE + V2 COMPARISON
# ================================================================
print("="*70)
print("📊 V5 IMPROVED vs V2 COMPARISON")
print("="*70)

# V5 Ensemble
weights_v5 = [0.4, 0.35, 0.25]
avg_proba_v5 = (weights_v5[0] * xgb_v5_proba + weights_v5[1] * lgb_v5_proba + weights_v5[2] * rf_v5_proba)
buy_prob_v5 = avg_proba_v5[:, 1] * 100

# Model agreement
agree_v5 = (xgb_v5_pred == 1) & (lgb_v5_pred == 1) & (rf_v5_pred == 1)
conf_v5 = buy_prob_v5.copy()
conf_v5[agree_v5] = np.minimum(conf_v5[agree_v5] + 5, 100)

print(f"\n📊 V5 IMPROVED BUY Signal Accuracy:")
print("-"*50)
for thresh in [70, 75, 80, 85, 90]:
    mask = conf_v5 >= thresh
    if mask.sum() > 0:
        signals = mask.sum()
        acc = y_test_v5[mask].mean() * 100
        print(f"   Conf >= {thresh}%: {signals:>5} signals | Accuracy: {acc:.1f}%")

# Compare with V2
print(f"\n" + "="*70)
print("📊 FINAL COMPARISON: V5 IMPROVED vs V2")
print("="*70)
print(f"\n{'Threshold':>12} | {'V2 Signals':>10} | {'V2 Acc':>8} | {'V5 Signals':>10} | {'V5 Acc':>8} | {'Winner':>8}")
print("-"*75)

for thresh in [70, 75, 80, 85, 90]:
    # V2 (from earlier cell)
    v2_mask = v2_confidence >= thresh
    v2_sig = v2_mask.sum()
    v2_acc = y_test_v2[v2_mask].mean() * 100 if v2_sig > 0 else 0
    
    # V5 Improved
    v5_mask = conf_v5 >= thresh
    v5_sig = v5_mask.sum()
    v5_acc = y_test_v5[v5_mask].mean() * 100 if v5_sig > 0 else 0
    
    winner = "V5 ✓" if v5_acc > v2_acc else ("V2 ✓" if v2_acc > v5_acc else "TIE")
    print(f"{thresh:>10}%+ | {v2_sig:>10} | {v2_acc:>7.1f}% | {v5_sig:>10} | {v5_acc:>7.1f}% | {winner:>8}")

📊 V5 IMPROVED vs V2 COMPARISON

📊 V5 IMPROVED BUY Signal Accuracy:
--------------------------------------------------
   Conf >= 70%:  3885 signals | Accuracy: 52.7%
   Conf >= 75%:  1053 signals | Accuracy: 58.7%
   Conf >= 80%:   274 signals | Accuracy: 55.5%
   Conf >= 85%:    83 signals | Accuracy: 59.0%
   Conf >= 90%:    10 signals | Accuracy: 30.0%

📊 FINAL COMPARISON: V5 IMPROVED vs V2

   Threshold | V2 Signals |   V2 Acc | V5 Signals |   V5 Acc |   Winner
---------------------------------------------------------------------------
        70%+ |       2344 |    51.8% |       3885 |    52.7% |     V5 ✓
        75%+ |        663 |    55.1% |       1053 |    58.7% |     V5 ✓
        80%+ |        112 |    58.9% |        274 |    55.5% |     V2 ✓
        85%+ |          3 |    33.3% |         83 |    59.0% |     V5 ✓
        90%+ |          0 |     0.0% |         10 |    30.0% |     V5 ✓


In [28]:
# ================================================================
# SAVE V5 IMPROVED MODELS (if better than V2)
# ================================================================
print("Saving V5 Improved models...")

joblib.dump(xgb_v5, MODEL_DIR / 'xgboost_v5.joblib')
joblib.dump(lgb_v5, MODEL_DIR / 'lightgbm_v5.joblib')
joblib.dump(rf_v5, MODEL_DIR / 'rf_v5.joblib')
joblib.dump(scaler_v5, MODEL_DIR / 'scaler_v5.joblib')
joblib.dump(feature_cols_v5, MODEL_DIR / 'feature_cols_v5.joblib')

config_v5 = {
    'version': 'v5_improved',
    'mode': 'BUY_vs_SELL',
    'min_confidence': 80,
    'weights': weights_v5,
    'labeling': 'V2 compatible (15 pips, 1.5x ratio)'
}
joblib.dump(config_v5, MODEL_DIR / 'config_v5.joblib')

print(f"✅ V5 Improved Models saved to {MODEL_DIR}")

Saving V5 Improved models...
✅ V5 Improved Models saved to c:\Users\Acer\Desktop\Forex-Signal-App\models\signal_generator_v5
✅ V5 Improved Models saved to c:\Users\Acer\Desktop\Forex-Signal-App\models\signal_generator_v5


In [30]:
# ================================================================
# FULL METRICS COMPARISON: V2 vs V5 (Same Test Data)
# ================================================================
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score

print("="*80)
print("📊 FULL METRICS COMPARISON: V2 vs V5")
print("   (Same test data: 80,296 BUY/SELL samples)")
print("="*80)

# V2 Ensemble predictions
v2_ensemble_pred = (v2_avg_proba[:, 1] > 0.5).astype(int)
v2_ensemble_proba = v2_avg_proba[:, 1]

# V5 Ensemble predictions  
v5_ensemble_pred = (avg_proba_v5[:, 1] > 0.5).astype(int)
v5_ensemble_proba = avg_proba_v5[:, 1]

print("\n" + "="*80)
print("📊 V2 MODEL METRICS (Test Data)")
print("="*80)
print(f"\n{'Model':<15} | {'Accuracy':>10} | {'Precision':>10} | {'Recall':>10} | {'F1-Score':>10} | {'ROC-AUC':>10}")
print("-"*80)

# V2 XGBoost
print(f"{'XGBoost':<15} | {accuracy_score(y_test_v2, v2_xgb_pred)*100:>9.1f}% | {precision_score(y_test_v2, v2_xgb_pred):>10.2f} | {recall_score(y_test_v2, v2_xgb_pred):>10.2f} | {f1_score(y_test_v2, v2_xgb_pred):>10.2f} | {roc_auc_score(y_test_v2, v2_xgb_proba[:, 1]):>10.2f}")

# V2 LightGBM
print(f"{'LightGBM':<15} | {accuracy_score(y_test_v2, v2_lgb_pred)*100:>9.1f}% | {precision_score(y_test_v2, v2_lgb_pred):>10.2f} | {recall_score(y_test_v2, v2_lgb_pred):>10.2f} | {f1_score(y_test_v2, v2_lgb_pred):>10.2f} | {roc_auc_score(y_test_v2, v2_lgb_proba[:, 1]):>10.2f}")

# V2 Random Forest
print(f"{'Random Forest':<15} | {accuracy_score(y_test_v2, v2_rf_pred)*100:>9.1f}% | {precision_score(y_test_v2, v2_rf_pred):>10.2f} | {recall_score(y_test_v2, v2_rf_pred):>10.2f} | {f1_score(y_test_v2, v2_rf_pred):>10.2f} | {roc_auc_score(y_test_v2, v2_rf_proba[:, 1]):>10.2f}")

# V2 Ensemble
print("-"*80)
print(f"{'ENSEMBLE V2':<15} | {accuracy_score(y_test_v2, v2_ensemble_pred)*100:>9.1f}% | {precision_score(y_test_v2, v2_ensemble_pred):>10.2f} | {recall_score(y_test_v2, v2_ensemble_pred):>10.2f} | {f1_score(y_test_v2, v2_ensemble_pred):>10.2f} | {roc_auc_score(y_test_v2, v2_ensemble_proba):>10.2f}")

print("\n" + "="*80)
print("📊 V5 MODEL METRICS (Test Data)")
print("="*80)
print(f"\n{'Model':<15} | {'Accuracy':>10} | {'Precision':>10} | {'Recall':>10} | {'F1-Score':>10} | {'ROC-AUC':>10}")
print("-"*80)

# V5 XGBoost
print(f"{'XGBoost':<15} | {accuracy_score(y_test_v5, xgb_v5_pred)*100:>9.1f}% | {precision_score(y_test_v5, xgb_v5_pred):>10.2f} | {recall_score(y_test_v5, xgb_v5_pred):>10.2f} | {f1_score(y_test_v5, xgb_v5_pred):>10.2f} | {roc_auc_score(y_test_v5, xgb_v5_proba[:, 1]):>10.2f}")

# V5 LightGBM
print(f"{'LightGBM':<15} | {accuracy_score(y_test_v5, lgb_v5_pred)*100:>9.1f}% | {precision_score(y_test_v5, lgb_v5_pred):>10.2f} | {recall_score(y_test_v5, lgb_v5_pred):>10.2f} | {f1_score(y_test_v5, lgb_v5_pred):>10.2f} | {roc_auc_score(y_test_v5, lgb_v5_proba[:, 1]):>10.2f}")

# V5 Random Forest
print(f"{'Random Forest':<15} | {accuracy_score(y_test_v5, rf_v5_pred)*100:>9.1f}% | {precision_score(y_test_v5, rf_v5_pred):>10.2f} | {recall_score(y_test_v5, rf_v5_pred):>10.2f} | {f1_score(y_test_v5, rf_v5_pred):>10.2f} | {roc_auc_score(y_test_v5, rf_v5_proba[:, 1]):>10.2f}")

# V5 Ensemble
print("-"*80)
print(f"{'ENSEMBLE V5':<15} | {accuracy_score(y_test_v5, v5_ensemble_pred)*100:>9.1f}% | {precision_score(y_test_v5, v5_ensemble_pred):>10.2f} | {recall_score(y_test_v5, v5_ensemble_pred):>10.2f} | {f1_score(y_test_v5, v5_ensemble_pred):>10.2f} | {roc_auc_score(y_test_v5, v5_ensemble_proba):>10.2f}")

print("\n" + "="*80)
print("⚠️  ТАНЫ ХЭЛСЭН V2 ҮЗҮҮЛЭЛТҮҮД (Accuracy 90%, ROC-AUC 0.87)")
print("    Энэ нь TRAIN DATA дээр эсвэл өөр TEST DATA дээр тооцоолсон байж магадгүй.")
print("    Дээрх үр дүнгүүд бол EUR_USD_test.csv дээрх бодит TEST үзүүлэлтүүд.")
print("="*80)

📊 FULL METRICS COMPARISON: V2 vs V5
   (Same test data: 80,296 BUY/SELL samples)

📊 V2 MODEL METRICS (Test Data)

Model           |   Accuracy |  Precision |     Recall |   F1-Score |    ROC-AUC
--------------------------------------------------------------------------------
XGBoost         |      49.6% |       0.54 |       0.25 |       0.34 |       0.51
LightGBM        |      49.6% |       0.55 |       0.18 |       0.28 |       0.51
Random Forest   |      51.4% |       0.55 |       0.40 |       0.46 |       0.53
--------------------------------------------------------------------------------
ENSEMBLE V2     |      49.6% |       0.54 |       0.23 |       0.32 |       0.51

📊 V5 MODEL METRICS (Test Data)

Model           |   Accuracy |  Precision |     Recall |   F1-Score |    ROC-AUC
--------------------------------------------------------------------------------
XGBoost         |      50.2% |       0.52 |       0.50 |       0.51 |       0.50
LightGBM        |      49.7% |       0.52 |

In [31]:
# ================================================================
# V2 BUY-ONLY BACKTEST (TP/SL based - REAL performance)
# ================================================================
print("="*80)
print("📊 V2 BUY-ONLY BACKTEST (TP=15 pips min move)")
print("   V2 BUY дохио өгөхөд TP хүрэх эсэхийг шалгах")
print("="*80)

# V2 test data-г TP/SL backtest хийх
# V2 BUY дохио = confidence >= threshold AND prediction = BUY (1)

def backtest_buy_signals(df, confidence, predictions, y_actual, min_conf_list=[70, 75, 80, 85, 90]):
    """
    Backtest BUY signals using TP/SL logic.
    BUY signal given when: confidence >= threshold AND predicted class = 1 (BUY)
    
    Success = price went UP by min_move_pips within forward_periods
    """
    results = {}
    
    for min_conf in min_conf_list:
        # BUY signals: high confidence + predicted BUY
        buy_mask = (confidence >= min_conf) & (predictions == 1)
        
        if buy_mask.sum() > 0:
            # How many of these were actually good BUY opportunities?
            # y_actual = 1 means it was labeled as BUY (up_move >= 15 pips, up > down * 1.5)
            correct = y_actual[buy_mask].sum()
            total = buy_mask.sum()
            accuracy = correct / total * 100
            
            results[min_conf] = {
                'signals': total,
                'correct': correct,
                'accuracy': accuracy
            }
    
    return results

# V2 Results
print("\n📊 V2 BUY Signal Performance (High Confidence Only):")
print("-"*60)
print(f"{'Confidence':>12} | {'Signals':>10} | {'Correct':>10} | {'Accuracy':>10}")
print("-"*60)

v2_ensemble_pred_class = (v2_avg_proba[:, 1] > 0.5).astype(int)
v2_results = backtest_buy_signals(test_binary_v2_clean, v2_confidence, v2_ensemble_pred_class, y_test_v2)

for conf, res in v2_results.items():
    print(f"{conf:>10}%+ | {res['signals']:>10} | {res['correct']:>10} | {res['accuracy']:>9.1f}%")

# V5 Results
print("\n📊 V5 BUY Signal Performance (High Confidence Only):")
print("-"*60)
print(f"{'Confidence':>12} | {'Signals':>10} | {'Correct':>10} | {'Accuracy':>10}")
print("-"*60)

v5_ensemble_pred_class = (avg_proba_v5[:, 1] > 0.5).astype(int)
v5_results = backtest_buy_signals(test_clean_v5, conf_v5, v5_ensemble_pred_class, y_test_v5)

for conf, res in v5_results.items():
    print(f"{conf:>10}%+ | {res['signals']:>10} | {res['correct']:>10} | {res['accuracy']:>9.1f}%")

# COMPARISON
print("\n" + "="*80)
print("📊 V2 vs V5 BUY-ONLY COMPARISON (High Confidence Signals)")
print("="*80)
print(f"\n{'Threshold':>12} | {'V2 Signals':>10} | {'V2 Acc':>8} | {'V5 Signals':>10} | {'V5 Acc':>8} | {'Winner':>8}")
print("-"*75)

for conf in [70, 75, 80, 85, 90]:
    v2_sig = v2_results.get(conf, {}).get('signals', 0)
    v2_acc = v2_results.get(conf, {}).get('accuracy', 0)
    v5_sig = v5_results.get(conf, {}).get('signals', 0)
    v5_acc = v5_results.get(conf, {}).get('accuracy', 0)
    
    if v2_acc > v5_acc:
        winner = "V2 ✓"
    elif v5_acc > v2_acc:
        winner = "V5 ✓"
    else:
        winner = "TIE"
    
    print(f"{conf:>10}%+ | {v2_sig:>10} | {v2_acc:>7.1f}% | {v5_sig:>10} | {v5_acc:>7.1f}% | {winner:>8}")

📊 V2 BUY-ONLY BACKTEST (TP=15 pips min move)
   V2 BUY дохио өгөхөд TP хүрэх эсэхийг шалгах

📊 V2 BUY Signal Performance (High Confidence Only):
------------------------------------------------------------
  Confidence |    Signals |    Correct |   Accuracy
------------------------------------------------------------
        70%+ |       2344 |       1214 |      51.8%
        75%+ |        663 |        365 |      55.1%
        80%+ |        112 |         66 |      58.9%
        85%+ |          3 |          1 |      33.3%

📊 V5 BUY Signal Performance (High Confidence Only):
------------------------------------------------------------
  Confidence |    Signals |    Correct |   Accuracy
------------------------------------------------------------
        70%+ |       3885 |       2047 |      52.7%
        75%+ |       1053 |        618 |      58.7%
        80%+ |        274 |        152 |      55.5%
        85%+ |         83 |         49 |      59.0%
        90%+ |         10 |          3

## 9. Save V5 Models

In [29]:
# Save V5 models
print("Saving V5 models...")

joblib.dump(xgb_model, MODEL_DIR / 'xgboost_v5.joblib')
joblib.dump(lgb_model, MODEL_DIR / 'lightgbm_v5.joblib')
joblib.dump(rf_model, MODEL_DIR / 'rf_v5.joblib')
joblib.dump(scaler, MODEL_DIR / 'scaler_v5.joblib')
joblib.dump(feature_cols, MODEL_DIR / 'feature_cols_v5.joblib')

# Save config
config = {
    'version': 'v5',
    'mode': 'BUY_ONLY',
    'min_confidence': 80,
    'tp_pips': 20,
    'sl_pips': 10,
    'weights': weights,
    'calibrated': True
}
joblib.dump(config, MODEL_DIR / 'config_v5.joblib')

print(f"✅ V5 Models saved to {MODEL_DIR}")
print(f"   - xgboost_v5.joblib (Calibrated)")
print(f"   - lightgbm_v5.joblib (Calibrated)")
print(f"   - rf_v5.joblib (Calibrated)")
print(f"   - scaler_v5.joblib")
print(f"   - feature_cols_v5.joblib")
print(f"   - config_v5.joblib")

Saving V5 models...
✅ V5 Models saved to c:\Users\Acer\Desktop\Forex-Signal-App\models\signal_generator_v5
   - xgboost_v5.joblib (Calibrated)
   - lightgbm_v5.joblib (Calibrated)
   - rf_v5.joblib (Calibrated)
   - scaler_v5.joblib
   - feature_cols_v5.joblib
   - config_v5.joblib
✅ V5 Models saved to c:\Users\Acer\Desktop\Forex-Signal-App\models\signal_generator_v5
   - xgboost_v5.joblib (Calibrated)
   - lightgbm_v5.joblib (Calibrated)
   - rf_v5.joblib (Calibrated)
   - scaler_v5.joblib
   - feature_cols_v5.joblib
   - config_v5.joblib
